<a href="https://colab.research.google.com/github/AdamLouly/AdamLouly.github.io/blob/master/Stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Price Prediction (AMAZON: AMZN)**
In this Notebook, we will use Linear Regression to predict stock prices thirty days into the future. This notebook is written in order to complete an academic project in Data Mining




In [0]:
!pip3 install quandl
import quandl #for stock Data
import pandas as pd
import numpy as np
import datetime

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, cross_validation, svm

  Running setup.py bdist_wheel for inflection ... - done
  Stored in directory: /root/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection


/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#**To get our stock data, we can set our dataframe to quandl.get("WIKI/[NAME OF STOCK]"). In this tutorial, I will use Amazon, but you can choose any stock you wish.**


In [0]:
df = quandl.get("WIKI/AMZN")
df.head(10)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1997-05-16,22.38,23.75,20.50,20.75,1225000.0,0.0,1.0,1.865000,1.979167,1.708333,1.729167,14700000.0
1997-05-19,20.50,21.25,19.50,20.50,508900.0,0.0,1.0,1.708333,1.770833,1.625000,1.708333,6106800.0
1997-05-20,20.75,21.00,19.63,19.63,455600.0,0.0,1.0,1.729167,1.750000,1.635833,1.635833,5467200.0
1997-05-21,19.25,19.75,16.50,17.13,1571100.0,0.0,1.0,1.604167,1.645833,1.375000,1.427500,18853200.0
1997-05-22,17.25,17.38,15.75,16.75,981400.0,0.0,1.0,1.437500,1.448333,1.312500,1.395833,11776800.0
1997-05-23,16.88,18.25,16.00,18.00,1328100.0,0.0,1.0,1.406667,1.520833,1.333333,1.500000,15937200.0
1997-05-27,17.75,19.75,17.50,19.00,724800.0,0.0,1.0,1.479167,1.645833,1.458333,1.583333,8697600.0
1997-05-28,19.31,19.63,18.38,18.38,381200.0,0.0,1.0,1.609167,1.635833,1.531667,1.531667,4574400.0
1997-05-29,18.50,18.50,17.75,18.06,289400.0,0.0,1.0,1.541667,1.541667,1.479167,1.505000,3472800.0


#We will only use simple linear regression on this example using Adj. Close variable
**An adjusted closing price is a stock's closing price on any given day of trading that has been amended to include any distributions and corporate actions that occurred at any time before the next day's open.
**

In [0]:
df = df[['Adj. Close']]

**We want to predict 30 days into the future, so we’ll set a variable pred equal to that. Then, we need to create a new column in our dataframe which serves as our label which will be the output of our model later
**

In [0]:
pred = int(30) # predicting 30 days into future
df['Prediction'] = df[['Adj. Close']].shift(-pred) #  label column with data shifted 30 units up
df.shape

(5248, 2)

# Defining Features & Labels
**Our X will be an array consisting of our Adj. Close values, and so we should drop the Prediction column. We also want to scale our input values. Scaling our features allow us to normalize the data.**

In [0]:
X = np.array(df.drop(['Prediction'], 1))
X = preprocessing.scale(X)

**Now, if you printed the dataframe after we created the Prediction column, you will find that for the last 30 days, there were NaNs, or no label data. We’ll set a new input variable to these days and remove them from the X array.**

In [0]:
df.tail()

,Adj. Close,Prediction
Date,,
2018-03-21,1581.86,NaN
2018-03-22,1544.10,NaN
2018-03-23,1495.56,NaN
2018-03-26,1555.86,NaN
2018-03-27,1497.05,NaN


In [0]:
X_forecast = X[-pred:] # set X_forecast equal to last 30
X = X[:-pred] # remove last 30 from X

**To define our y, or output, we will set it equal to our array of the Prediction values and remove the last 30 days where we don’t have any pricing data.**

In [0]:
y = np.array(df['Prediction'])
y = y[:-pred]

# Model Development : Linear Regression
**
Our data is prepared now, we should now split data into training/test data in order to feed it to our model.**

In [0]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size = 0.2)

**Now, we can initiate our Linear Regression model and fit it with training data. After training, to test the accuracy of the model, we “score” it using the testing data. We can get an r^2 (coefficient of determination) reading based on how far the predicted price was compared to the actual price in the test data set.**

In [0]:
# Training
clf = LinearRegression()
clf.fit(X_train,y_train)
# Testing
confidence = clf.score(X_test, y_test)
print("confidence: ", confidence)

confidence:  0.9866876730946079


**Lastly, we can to predict our X_forecast values:
**

In [0]:
forecast_prediction = clf.predict(X_forecast)
print(forecast_prediction)

[1500.22266703 1539.06677971 1550.45212308 1536.55796455 1557.45767049
 1572.94641493 1574.90244031 1591.10343312 1614.43754022 1603.83885923
 1604.33849614 1584.140408   1591.36919744 1616.20221529 1631.1169088
 1638.94101031 1646.23358319 1674.96802122 1695.69763801 1684.84382327
 1687.84164478 1678.61430766 1667.3033783  1638.8665963  1683.06851763
 1678.12530132 1637.98425877 1586.38345884 1650.48581227 1587.96741417]
